<a href="https://colab.research.google.com/github/badjuju0/miigaikPractice/blob/main/Zadanie4/%D0%9D%D0%B5%D0%B4%D0%B5%D0%BB%D1%8F_4_(2025).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Неделя 4. Пространственные запросы и пространтсвенные операции
Задание этой недели будет похоже на то, что мы делали на неделе 1 (мы тоже будем парсить датасеты с геоданными) и на то, что мы делали на неделе 2 (визуализация на карте пригодится также).


## Знакомство с датасетом регионов
Для начала посмотрим, что содержится в файле russian_regions.csv

In [1]:
import pandas as pd

df = pd.read_csv('russian_regions.csv', sep=';', encoding='cp1251')
df.head()


,Регион,Полигон,ФО,Unnamed: 3
0,Белгородская область,"[[[50.97531, 35.352225], [50.958115, 35.411367...",Центральный федеральный округ,NaN
1,Брянская область,"[[[53.911337, 34.146682], [53.908911, 34.14098...",Центральный федеральный округ,NaN
2,Владимирская область,"[[[56.524305, 38.268605], [56.526572, 38.27968...",Центральный федеральный округ,NaN
3,Воронежская область,"[[[51.637092, 38.140099], [51.670452, 38.16122...",Центральный федеральный округ,NaN
4,Ивановская область,"[[[56.771178, 39.400991], [56.779082, 39.40670...",Центральный федеральный округ,NaN


Посмотрим, как обращаться со столбцом "Полигон". Для примера выведем на карту границы Московской области

In [2]:

import pandas as pd
import shapely
import ast
import folium

# Функция парсинга координат
#Преобразование координат в формат списка
def parse_coordinates(coord_str):
    try:
        return (ast.literal_eval(coord_str))[0]
    except:
        return None

df = pd.read_csv('russian_regions.csv', sep=';', encoding='cp1251')
df1 = df[df['Регион'] == "Московская область" ]

for idx, row in df1.iterrows():
    coords = parse_coordinates(row['Полигон'])
    # Создаем полигон с помощью библиотеки Shapely
    polygon = shapely.Polygon(coords)

if coords:
  # Получаем координаты центроида
  centroid = polygon.centroid.coords[0]

  # Координаты центроида
  lat_centroid, lon_centroid = centroid
  m = folium.Map(location=[lat_centroid, lon_centroid], zoom_start=8, tiles='Cartodb Positron',)
  folium.Polygon(
                locations=coords,
                tooltip=row['Регион'],
                color='purple',
                weight=0.5,
                fill = True,
            ).add_to(m)
m


## Знакомство с другим датасетом

In [5]:
import pandas as pd


df3 = pd.read_csv('objects_v2.csv', sep=';', encoding='utf-8')

df3.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 23: unexpected end of data

#Совместим данные из разных датасетов на одной карте
Что полезного есть в коде ниже:
- метод MarkerCluster из плагинов для карты folium. Позволяет красиво объединить точечные маркеры, для аккуратного просмотра карты. Почитать о нем можно [тут](https://newtechaudit.ru/vizualiziruj-prosto-interaktivnaya-karta-cherez-folium/ ) или [здесь](https://proglib.io/p/python-web-map)
- метод shapely.contains проверяет вхождение в полигон точки.


In [4]:
from folium.plugins import MarkerCluster

marker_cluster = MarkerCluster().add_to(m)
#найти все точки, которые попадают в "наш" полигон и при этом в названии содержат слово "Завод"

for cur_row in df3.iterrows():
  cur_pos = shapely.Point([float(cur_row[1]["latitude"]), float(cur_row[1]["longitude"])])
  if shapely.contains(polygon, cur_pos) and "Завод " in cur_row[1]["name"]:
    folium.Marker(
        location = [float(cur_row[1]["latitude"]), float(cur_row[1]["longitude"])],
        popup=cur_row[1]["address"], tooltip=cur_row[1]["name"]
    ).add_to(marker_cluster)

m

NameError: name 'df3' is not defined

#Задание
1. По аналогии с прошлыми заданиями результат должен быть представлен в result.html. На этот раз заготовки нет и вам придется собрать ее самим. Не забывайте про название работы, ФИО и группу.
2. Из датасета russian_regions загрузите свой регион, его границы добавьте на карту, сделайте заливку полигона цветом на ваш выбор.
3. Выберите несколько (не менее трех) улиц, которые есть в вашем регионе. Из второго датасета отфильтруйте все объекты, чей адрес содержит одну из этих улиц. Добавьте эти объекты на карту.
4. Не забудьте про popup (с адресом), tooltip (с названием) и MarkerCluster (для визуального объединения кластеров).
5. В result должна быть, помимо заголовочных данных, ОДНА карта с перечисленным в предыдущих пунктах содержимым.

In [6]:
import pandas as pd
import shapely
import ast
import folium
from folium.plugins import MarkerCluster

result_html='''<html>
    <body>
      <h1>Отчет за 4 неделю ознакомительной практики по разработке программного обеспечения геосервиса</h1>
      <h2>Выполнил: студент группы 2024-ИСиТ-1см: Горбачев П.А.</h2>'''

def parse_coordinates(coord_str):
    try:
        return (ast.literal_eval(coord_str))[0]
    except:
        return None

# Полигон региона "г. Москва" отображается неверно на карте, поэтому выберу "Тамбовская область"
df = pd.read_csv('russian_regions.csv', sep=';', encoding='cp1251')
df1 = df[df['Регион'] == "Курская область" ]

for idx, row in df1.iterrows():
    coords = parse_coordinates(row['Полигон'])
    polygon = shapely.Polygon(coords)

if coords:
  centroid = polygon.centroid.coords[0]
  lat_centroid, lon_centroid = centroid
  m = folium.Map(location=[lat_centroid, lon_centroid], zoom_start=8, tiles='Cartodb Positron',)
  folium.Polygon(
                locations=coords,
                tooltip=row['Регион'],
                color='red',
                weight=0.5,
                fill = True,
            ).add_to(m)

df3 = pd.read_csv('objects_v2.csv', sep=';', encoding='utf-8')

marker_cluster = MarkerCluster().add_to(m)

for cur_row in df3.iterrows():
    cur_pos = shapely.Point([float(cur_row[1]["latitude"]), float(cur_row[1]["longitude"])])
    address = cur_row[1]["address"]

    if (shapely.contains(polygon, cur_pos) and
        isinstance(address, str) and
        ("Золотая" in address or
         "Ватутина" in address or
         "Радищева" in address)):
        folium.Marker(location=[float(cur_row[1]["latitude"]), float(cur_row[1]["longitude"])],
            popup=address, tooltip=cur_row[1]["name"]
        ).add_to(marker_cluster)

map_html = m.get_root().render()

result_html += f'''
      <h3>Карта объектов</h3>
      {map_html}
  </html>'''
file_html = open("result_4.html", "w")
file_html.write(result_html)
file_html.close()


/tmp/ipython-input-310801635.py:38: DtypeWarning: Columns (16,17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('objects_v2.csv', sep=';', encoding='utf-8')
